In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from xgboost import XGBRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def train_test_data(trainPath, testPath):
    test = pd.read_csv(testPath).set_index('id') 
    train = pd.read_csv(trainPath).set_index('id')
    return train, test

def remplazar_faltantes(X):
    imputer = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
    return imputer.fit_transform(X)

def Scalar(X):
    return preprocessing.StandardScaler().fit_transform(X)

def Normalizar(X):
    return preprocessing.normalize(X,norm='l1')

def PreProcesamiento(trainPath,testPath):
    train, test = train_test_data(trainPath, testPath)
    
    y_train = train.duration.values
    X_train = train.drop('duration',axis=1)
    
    X_test = test.copy()
    
    X_train = remplazar_faltantes(X_train)
    X_train = Scalar(X_train)
    X_train = Normalizar(X_train)
    
    X_test = remplazar_faltantes(X_test)
    X_test = Scalar(X_test)
    X_test = Normalizar(X_test)
    
    return X_train, y_train, X_test    

In [3]:
trainPath = "../Data/train.csv"
testPath = "../Data/test.csv"

X_train, y_train, X_test =  PreProcesamiento(trainPath,testPath)

In [4]:
xgboost = XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
                      gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=16,
                      min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
                      objective='reg:linear', reg_alpha=1e-05, reg_lambda=1,
                      scale_pos_weight=1, seed=25, silent=True, subsample=0.9)

In [5]:
xgboost = xgboost.fit(X_train,y_train)

In [6]:
y_pred_xg = xgboost.predict(X_test)

In [17]:
test = pd.read_csv(testPath)

In [18]:
df = pd.DataFrame({"id": list(test.id), 'duration':y_pred_xg})
df = df.set_index('id')
df.to_csv("~/sampleSubmission.csv")